In [24]:
import socket  
import numpy as np
import math
from numpy.linalg import norm
from math import *
from matplotlib import pyplot as plt
from matplotlib.patches import Polygon
from random import random
from scipy.spatial import ConvexHull
from matplotlib import path
import time
from random import random
import keyboard ### NOTE: Run first: !pip install keyboard    in the terminal to install keyboard module
from IPython.display import clear_output

start = False
n = 0 # To show the first READY! message
index = 0 # To select the coordinates from the array

################################################################################################# STARTS COORD VISION TO NORM
def coordinatesfromvisiontonormal (XR1OPP,YR1OPP,XR2OPP,YR2OPP,XR3OPP,YR3OPP,XR1,YR1,XR2,YR2,XR3,YR3,Xball,Yball):
    XR1OPP = ((XR1OPP*10)/75)-25
    YR1OPP = abs((((YR1OPP*10)/75)-(5/3))-130)
    XR2OPP = ((XR2OPP*10)/75)-25
    YR2OPP = abs((((YR2OPP*10)/75)-(5/3))-130)
    XR3OPP = ((XR3OPP*10)/75)-25
    YR3OPP = abs((((YR3OPP*10)/75)-(5/3))-130)
    XR1 = ((XR1*10)/75)-25 #SET AS S
    YR1 = abs((((YR1*10)/75)-(5/3))-130)
    XR2 = ((XR2*10)/75)-25 #SET AS D
    YR2 = abs((((YR2*10)/75)-(5/3))-130)
    XR3 = ((XR3*10)/75)-25 #SET AS G
    YR3 = abs((((YR3*10)/75)-(5/3))-130)
    Xball = ((Xball*10)/75)-25
    Yball = abs((((Yball*10)/75)-(5/3))-130)
    return XR1OPP,YR1OPP,XR2OPP,YR2OPP,XR3OPP,YR3OPP,XR1,YR1,XR2,YR2,XR3,YR3,Xball,Yball
################################################################################################# ENDS COORD VISION TO NORM
###################################################################################################### STARTS GOES FOR THE BALL
def GOESFORTHEBALL(Xball,Yball,XR3,YR3,XR2,YR2,XR1,YR1):
    R3_2ball = math.sqrt((Xball-XR3)**2 + (Yball-YR3)**2)
    R2_2ball = math.sqrt((Xball-XR2)**2 + (Yball-YR2)**2)
    R1_2ball = math.sqrt((Xball-XR1)**2 + (Yball-YR1)**2)
    if R3_2ball <= R2_2ball:
        if R3_2ball <= R1_2ball:
            return 3
        else:
            return 1
    else:
        if R2_2ball <= R1_2ball:
            return 2
        else:
            return 1       
###################################################################################################### ENDS GOES FOR THE BALL
###################################################################################################### STARTS PATH PLANNING
def pathplanningRRT (Xobs1, Yobs1, Xobs2, Yobs2, Xobs3, Yobs3, Xobs4, Yobs4, Xobs5, Yobs5, Xstart, Ystart, Xgoal, Ygoal):
    
    def isCollisionFreeVertex(obstacles, xy):
        collFree = True

        for obstacle in obstacles:
            hull = path.Path(obstacle)
            collFree = not hull.contains_points([xy])
            if hull.contains_points([xy]):
                return False
        return collFree

    def isCollisionFreeEdge(obstacles, closest_vert, xy):
        closest_vert = np.array(closest_vert); xy = np.array(xy)
        collFree = True
        l = norm(closest_vert - xy)
        map_resolution = 0.01; M = int(l / map_resolution)
        if M <= 2: M = 20
        t = np.linspace(0,1,M)
        for i in range(1,M-1):
            p = (1-t[i])*closest_vert + t[i]*xy # COMPUTES CONFIGURATION
            collFree = isCollisionFreeVertex(obstacles, p) 
            if collFree == False: return False
        return collFree

    def draw_map(obstacles):

        # Bounds on world
        world_bounds_x = [-0, 150]
        world_bounds_y = [-0, 130]

        # Draw obstacles
        #fig = plt.figure(figsize=(10, 10))
        #plt.grid()
        #ax = plt.gca()
        #ax.set_xlim(world_bounds_x)
        #ax.set_ylim(world_bounds_y)
        #for k in range(len(obstacles)):
            #ax.add_patch( Polygon(obstacles[k]) )
        
    obstacles = [
                 np.array([[Xobs1-5, Yobs1-5], [Xobs1-5, Yobs1+5], [Xobs1+5, Yobs1+5], [Xobs1+5, Yobs1-5]]),
                 np.array([[Xobs2-5, Yobs2-5], [Xobs2-5, Yobs2+5], [Xobs2+5, Yobs2+5], [Xobs2+5, Yobs2-5]]),
                 np.array([[Xobs3-5, Yobs3-5], [Xobs3-5, Yobs3+5], [Xobs3+5, Yobs3+5], [Xobs3+5, Yobs3-5]]), 
                 np.array([[Xobs4-5, Yobs4-5], [Xobs4-5, Yobs4+5], [Xobs4+5, Yobs4+5], [Xobs4+5, Yobs4-5]]),
                 np.array([[Xobs5-5, Yobs5-5], [Xobs5-5, Yobs5+5], [Xobs5+5, Yobs5+5], [Xobs5+5, Yobs5-5]]),
                ]
    draw_map(obstacles)

    class Node:
        def __init__(self):
            self.p     = [0, 0]
            self.i     = 0
            self.iPrev = 0
            
    def closestNode(rrt, p):
        distance = []
        for node in rrt:
            distance.append( sqrt((p[0] - node.p[0])**2 + (p[1] - node.p[1])**2) )
        distance = np.array(distance)
    
        dmin = min(distance)
        ind_min = distance.tolist().index(dmin)
        closest_node = rrt[ind_min]

        return closest_node

    def SmoothPath(P, obstacles, smoothiters):
    
        m = P.shape[0]
        l = np.zeros(m)
        for k in range(1, m):
            l[k] = norm(P[k,:]-P[k-1,:]) + l[k-1] # FINDS ALL THE STRAIGTH-LINE DISTANCES
        iters = 0
        while iters < smoothiters:
            s1 = random()*l[m-1] 
            s2 = random()*l[m-1]
            if s2 < s1:
                temps = s1
                s1 = s2
                s2 = temps
            for k in range(1, m):
                if s1 < l[k]:
                    i = k - 1
                    break
            for k in range(i, m):
                if s2 < l[k]:
                    j = k - 1
                    break
            if (j <= i):
                iters = iters + 1
                continue
            t1 = (s1 - l[i]) / (l[i+1]-l[i])
            gamma1 = (1 - t1)*P[i,:] + t1*P[i+1,:]
            t2 = (s2 - l[j]) / (l[j+1]-l[j])
            gamma2 = (1 - t2)*P[j,:] + t2*P[j+1,:]
        
            collisionFree = isCollisionFreeEdge(obstacles, gamma1, gamma2)
            if collisionFree == 0:
                iters = iters + 1
                continue

            P = np.vstack([P[:(i+1),:], gamma1, gamma2, P[(j+1):m,:]])
            m = P.shape[0]
            l = np.zeros(m)
            for k in range(1, m):
                l[k] = norm( P[k,:] - P[k-1,:] ) + l[k-1]
            iters = iters + 1

        P_smooth = P
    
        return P_smooth

# Initialization
    visualize = 1
    maxiters  = 5000
    nearGoal = False # This will be set to true if goal has been reached
    minDistGoal = 0.25 # Convergence criterion: success when the tree reaches within 0.25 in distance from the goal.
    d = 0.20 # Extension parameter: this controls how far the RRT extends in each step.

    draw_map(obstacles)

# Start and goal positions
    xy_start = np.array([Xstart, Ystart]);# plt.plot(xy_start[0],xy_start[1],'bo',color='red', markersize=20)
    xy_goal =  np.array([Xgoal, Ygoal]); # plt.plot(xy_goal[0], xy_goal[1], 'bo',color='green',markersize=20)
# The RRT is a 2 x N list of points.
# Each column represents a vertex of the tree.
    rrt = []
    start_node = Node()
    start_node.p = xy_start
    start_node.i = 0
    start_node.iPrev = 0
    rrt.append(start_node)


# RRT algorithm
    start_time = time.time()
    iters = 0
    while not nearGoal and iters < maxiters:
        # Sample point
        rnd = random()
        # With probability 0.05, sample the goal. This promotes movement to the goal.
        if rnd < 0.05:
            xy = xy_goal
        else:
            # Sample (uniformly) from space (with probability 0.95). The space is defined
            # with the bounds world_bounds_x and world_bounds_y defined above.
            xy = np.array([random()*150, random()*150]) 
        # Check if sample is collision free
        collFree = isCollisionFreeVertex(obstacles, xy)
        # If it's not collision free, continue with loop
        if not collFree:
            iters += 1
            continue

        # If it is collision free, find closest point in existing tree. 
        closest_node = closestNode(rrt, xy)
    
        # Extend tree towards xy from closest_vert. Parameter d is step size. Thus, the Euclidean
        # distance between new_vert and closest_vert should be d.
        new_node = Node()
        new_node.p = closest_node.p + d * (xy - closest_node.p)
        new_node.i = len(rrt)
        new_node.iPrev = closest_node.i

        # Check if new vertice is in collision
        collFree = isCollisionFreeEdge(obstacles, closest_node.p, new_node.p)
        # If it's not collision free, continue with loop
        if not collFree:
            iters += 1
            continue
    
        # If it is collision free, add it to tree    
        rrt.append(new_node)

        if visualize:
            c = 0;
            # plt.plot(xy[0], xy[1], 'ro', color='k')
            #plt.plot(new_node.p[0], new_node.p[1], 'bo',color = 'magenta', markersize=5) # VERTICES
            #plt.plot([closest_node.p[0], new_node.p[0]], [closest_node.p[1], new_node.p[1]], color='blue') # EDGES
    
        # Check if we have reached the goal
        if norm(np.array(xy_goal) - np.array(new_node.p)) < minDistGoal:
            # Add last, goal node
            goal_node = Node()
            goal_node.p = xy_goal
            goal_node.i = len(rrt)
            goal_node.iPrev = new_node.i
            if isCollisionFreeEdge(obstacles, new_node.p, goal_node.p):
                rrt.append(goal_node)
                P = [goal_node.p]
            else: P = []
            end_time = time.time()
            nearGoal = True
            print ("Reached the goal after %.2f seconds:" % (end_time - start_time))

        iters += 1

    print ('Number of iterations passed: %d / %d' %(iters, maxiters))
    print ('RRT length: ', len(rrt))

# Path construction from RRT:
    print ('Constructing the path...')
    i = len(rrt) - 1
    while True:
        i = rrt[i].iPrev
        P.append(rrt[i].p)
        if i == 0:
            print ('Reached RRT start node')
            break
    P = np.array(P)
    #plt.plot( P[:,0], P[:,1], color='green', linewidth=5, label='RRT path' )

# Path smoothing:
    print ('Path smoothing...')
    P_smooth = SmoothPath(P, obstacles, smoothiters = 10)
    #plt.plot(P_smooth[:,0], P_smooth[:,1], linewidth=5, color='orange', label='smoothed path')

### reversed points for pic
    A = np.abs(P_smooth+[0,-130])
    WCONV = ((A*75)/10)+[187.5,12.5]
    #print (WCONV)
    
    return WCONV
###################################################################################################### STARTS PATH PLANNING
################################################################################################### ENDS SECTION FUNCTIONS


#################################################### Needed
side = 0 # 0 = LEFT / 1 = RIGHT

## THESE COME FROM THE REFEREE
SIDE = 0  # 0 = LEFT / 1 = RIGHT
BALLPOSSE = 0 # DECIDED BY TOSSING A COIN
KICKOFF = 0 # IF KICKOFF = 1 THERE IS A KICKOFF

####################################################


#### STARTS "MAIN"

HOST = 'localhost'
PORT = 12345
s = socket.socket()
s.connect((HOST, PORT))
print("Data from Python:")
change = 1
stop = False
#################### START DATA RECEIVE ###############################3"
########################################################################################################## STARTS REC DATA 
while True: 
    data = s.recv(1000)
    data2 = data.decode()
    arr = data2.split()
    
    if len(arr)>20:
        arr = arr[arr.index('s')+1:]
        arr = arr[:arr.index('\x00')]
        data = ""
        # BALL REAL TIME
        Xball = float (arr[0])
        Yball = float (arr[1])
        
        ################ PLAYERS FROM OWN TEAM #####################################################################################################################################
        # GOALIE (OURS) REAL TIME
        XR3 = float(arr[2])
        YR3 = float(arr[3])
        Ar3 = float(arr[4])
        # MID - DEF (OURS) REAL TIME
        XR2 = float(arr[5])
        YR2 = float(arr[6])
        Ar2 = float(arr[7])
        # STRIKER (OURS) REAL TIME
        XR1 = float(arr[8])
        YR1 = float(arr[9])
        Ar1 = float(arr[10])
        ################ PLAYERS FROM OWN TEAM #####################################################################################################################################
        # GOALIE (OPP) REAL TIME
        XR1OPP = float(arr[11])
        YR1OPP = float(arr[12])
        # MID - DEF (OPP) REAL TIME
        XR2OPP = float(arr[14])
        YR2OPP = float(arr[15])
        # STRIKER (OPP) REAL TIME
        XR3OPP = float(arr[17])
        YR3OPP = float(arr[18])
        # 1350, 500
#         m = (Yball-500)/(Xball-1350)
#         alpha = atan(m)
#         nXball = cos(alpha) * acos(10+10*)
#         nYball = sin(alpha) * acos(10+10*)
        
        if n == 0:
            print("READY!")
        n = n+1
        
        ############################# SEND DATA TO PROCESSING SERVER ##################################################
        if keyboard.is_pressed('ENTER'):
            start = True
            print("Key pressed")

####################################################################################################################################################################           

        if start == True:
            case = GOESFORTHEBALL(Xball,Yball,XR3,YR3,XR2,YR2,XR1,YR1)
            #case = 3
            # coordinates = pathplanningRRT (Xobs1, Yobs1, Xobs2, Yobs2, Xobs3, Yobs3, Xobs4, Yobs4, Xobs5, Yobs5, Xstart, Ystart, Xgoal, Ygoal)
            if (change == 1) and (stop == False):
                XR1OPP,YR1OPP,XR2OPP,YR2OPP,XR3OPP,YR3OPP,XR1,YR1,XR2,YR2,XR3,YR3,Xball,Yball = coordinatesfromvisiontonormal (XR1OPP,YR1OPP,XR2OPP,YR2OPP,XR3OPP,YR3OPP,XR1,YR1,XR2,YR2,XR3,YR3,Xball,Yball)
                stop = True
                if case == 3: # S GOES FOR THE BALL

                    ############################### S GOES FOR THE BALL R3
                    Xobs1 = XR1
                    Yobs1 = YR1
                    Xobs2 = XR2
                    Yobs2 = YR2
                    Xobs3 = XR1OPP
                    Yobs3 = YR1OPP
                    Xobs4 = XR2OPP
                    Yobs4 = YR2OPP
                    Xobs5 = XR3OPP
                    Yobs5 = YR3OPP
                    Xgoal = Xball
                    Ygoal = Yball
                    Xstart = XR3
                    Ystart = YR3
                    WCONV =  pathplanningRRT (Xobs1, Yobs1, Xobs2, Yobs2, Xobs3, Yobs3, Xobs4, Yobs4, Xobs5, Yobs5, Xstart, Ystart, Xgoal, Ygoal)

    #                 ################################ D MOVES R2
    #                 Xobs1 = XR1
    #                 Yobs1 = YR1
    #                 Xobs2 = XR3
    #                 Yobs2 = YR3
    #                 Xobs3 = XR1OPP
    #                 Yobs3 = YR1OPP
    #                 Xobs4 = XR2OPP
    #                 Yobs4 = YR2OPP
    #                 Xobs5 = XR3OPP
    #                 Yobs5 = YR3OPP
    #                 if Xball < 45:
    #                     Xball = Xball + 40 
    #                 if Xball > 85:
    #                     Xball = Xball - 40
    #                 if Yball < 45:
    #                     Yball = Yball +40 
    #                 if Yball > 105:
    #                     Yball = Yball - 40
    #                 Xgoal = Xball
    #                 Ygoal = Yball
    #                 Xstart = XR2
    #                 Ystart = YR2
    #                 WCONV2 = pathplanningRRT (Xobs1, Yobs1, Xobs2, Yobs2, Xobs3, Yobs3, Xobs4, Yobs4, Xobs5, Yobs5, Xstart, Ystart, Xgoal, Ygoal)

    #                 ################################# G ANGLE
    #                 if (side == 0): # LEFT
    #                     GLXours = 0;
    #                     GLYours = 65;

    #                 if (side == 1): # RIGHT
    #                     GLXours = 150;
    #                     GLYours = 65;

    #                 Ar1desired = np.rad2deg(np.arctan((Yball - GLYours)/(Xball-GLXours)))

                if case == 1: # G GOES FOR THE BALL
                    Xobs1 = XR3
                    Yobs1 = YR3
                    Xobs2 = XR2
                    Yobs2 = YR2
                    Xobs3 = XR1OPP
                    Yobs3 = YR1OPP
                    Xobs4 = XR2OPP
                    Yobs4 = YR2OPP
                    Xobs5 = XR3OPP
                    Yobs5 = YR3OPP
                    Xgoal = Xball
                    Ygoal = Yball
                    Xstart = XR1
                    Ystart = YR1
                    WCONV = pathplanningRRT (Xobs1, Yobs1, Xobs2, Yobs2, Xobs3, Yobs3, Xobs4, Yobs4, Xobs5, Yobs5, Xstart, Ystart, Xgoal, Ygoal)

                if case == 2: # D GOES FOR THE BALL

                    ############################### S GOES FOR THE BALL R3
                    Xobs1 = XR1
                    Yobs1 = YR1
                    Xobs2 = XR3
                    Yobs2 = YR3
                    Xobs3 = XR1OPP
                    Yobs3 = YR1OPP
                    Xobs4 = XR2OPP
                    Yobs4 = YR2OPP
                    Xobs5 = XR3OPP
                    Yobs5 = YR3OPP
                    Xgoal = Xball
                    Ygoal = Yball
                    Xstart = XR2
                    Ystart = YR2
                    WCONV =  pathplanningRRT (Xobs1, Yobs1, Xobs2, Yobs2, Xobs3, Yobs3, Xobs4, Yobs4, Xobs5, Yobs5, Xstart, Ystart, Xgoal, Ygoal)

                    ################################ D MOVES R2
    #                 Xobs1 = XR1
    #                 Yobs1 = YR1
    #                 Xobs2 = XR2
    #                 Yobs2 = YR2
    #                 Xobs3 = XR1OPP
    #                 Yobs3 = YR1OPP
    #                 Xobs4 = XR2OPP
    #                 Yobs4 = YR2OPP
    #                 Xobs5 = XR3OPP
    #                 Yobs5 = YR3OPP
    #                 if Xball < 45:
    #                     Xball = Xball + 40 
    #                 if Xball > 85:
    #                     Xball = Xball - 40
    #                 if Yball < 45:
    #                     Yball = Yball +40 
    #                 if Yball > 105:
    #                     Yball = Yball - 40
    #                 Xgoal = Xball
    #                 Ygoal = Yball
    #                 Xstart = XR3
    #                 Ystart = YR3
    #                 WCONV2 = pathplanningRRT (Xobs1, Yobs1, Xobs2, Yobs2, Xobs3, Yobs3, Xobs4, Yobs4, Xobs5, Yobs5, Xstart, Ystart, Xgoal, Ygoal)

                    ################################# G ANGLE
    #                 if (side == 0): # LEFT
    #                     GLXours = 0;
    #                     GLYours = 65;

    #                 if (side == 1): # RIGHT
    #                     GLXours = 150;
    #                     GLYours = 65;

    #                 Ar1desired = np.rad2deg(np.arctan((Yball - GLYours)/(Xball-GLXours)))

                # Case 1, 2 o 3
                #XR1OPP,YR1OPP,XR2OPP,YR2OPP,XR3OPP,YR3OPP,XR1,YR1,XR2,YR2,XR3,YR3,Xball,Yball = coordinatesfromvisiontonormal (XR1OPP,YR1OPP,XR2OPP,YR2OPP,XR3OPP,YR3OPP,XR1,YR1,XR2,YR2,XR3,YR3,Xball,Yball)
    #             DS_GOP, DS_OP1, DS_OP2, DSB, DC, DSD, DSG, DD_GOP, DD_OP1, DD_OP2, DDB, DC2, DDS, DDG, DG_GOP, DG_OP1,DG_OP2, DGB, DC3 = distances (XR1OPP,YR1OPP,XR2OPP,YR2OPP,XR3OPP,YR3OPP,XR1,YR1,XR2,YR2,XR3,YR3,Xball,Yball,0)
    #             case = GOESFORTHEBALL (DS_GOP, DS_OP1, DS_OP2, DSB, DC, DSD, DSG, DD_GOP, DD_OP1, DD_OP2, DDB, DC2, DDS, DDG, DG_GOP, DG_OP1,DG_OP2, DGB, DC3)
                # print(case)
                #coordinates = [[640,400],[680,390],[690,360],[700,340],[720,300],[740,300],[760,300],[780,300],[800,300],[850,300]]
                #coordinates = [[640,450],[680,460],[690,490],[700,510],[720,550],[740,550],[760,550],[780,550],[800,550],[850,550]]
                coordinates = np.floor(WCONV)
                coordinates = np.flip(coordinates,0)
                #coordinates = [[749.,499.], [750.,497.], [750.,496.], [751.,494.], [751.,491.], [753.,487.], [761.,480.], [805.,450.], [853.,419.]]
                print(coordinates)
                #coordinates = [[630,450],[620,460],[610,490],[600,510],[580,550],[560,550],[540,550],[520,550],[500,550],[450,550]]
                #coordinates = [[629, 424], [638,425], [741, 490], [752, 497], [753, 498], [752, 498],[751,498],[751,498],[751,498],[749,499]]
                ############################# TARGET ANGLE #######################
                ####### NOTA! Para cada punto de la trayectoria del path planning, x1 y y1 son las posiciones actuales del robot, 
                # es decir: XR3 y YR3 para el primer robot. Mientras que x2 y y2 son las siguientes coordenadas a seguir de la trayectoria
                # Mientras el robot avanza por la trayectoria que le digamos, x1 y y1 se actualizan a lo que envíe el sistema de visión y
                # x2 y y2 se actualizan al siguiente punto de la trayectoria
            
            # x1 and y1 are the robot points
#             x1 = XR3
#             y1 = YR3
            if case == 1:
                x1 = XR1
                y1 = YR1
                Robot_angle = Ar1
            if case == 2:
                x1 = XR2
                y1 = YR2
                Robot_angle = Ar2
            if case == 3:
                x1 = XR3
                y1 = YR3
                Robot_angle = Ar3
            
            # x2 and y2 are the goal points  ###### HERE THE TARGET COORDINATES ARE MANUALLY SET #########
            x2 = coordinates[index][0]
            y2 = coordinates[index][1]
            if index < (len(coordinates)-1):
                index = index+1
            
            if x2 != x1:
                slope = (y1-y2)/(x2-x1)
                theta = math.degrees(math.atan(slope))
                if theta>0:
                    if (x2-x1)<0:
                        theta = theta + 180.0
                elif theta<0:
                    if (x2-x1)<0:
                        theta = 180.0 + theta
                    else:
                        theta = 360.0 + theta
                else:
                    if (x2-x1)<0:
                        theta = 180.0
            else:
                if y1 > y2:
                    theta = 90.0
                else:
                    theta = 270.0

            if Robot_angle<180:
                limit = Robot_angle + 180
                if theta > limit:
                    angle_error = -(360-(theta-Robot_angle))
                else:
                    angle_error = (theta-Robot_angle)
            elif (Robot_angle)>180:
                limit = Robot_angle - 180
                if theta < limit:
                    angle_error = 360+(theta-Robot_angle)
                else:
                    angle_error = (theta-Robot_angle)
            else:
                angle_error = (theta-Robot_angle)

            ka = 0.6  # Gain for the angular velocity controller (Adjust if the robot spins too fast or too slow)
            angle = ka * angle_error
            kv = 0.07 # Gain for the linear velocity controller (Adjust if the robot goes too fast or too slow)
            LV = kv*math.sqrt((x2-x1)**2 + (y2-y1)**2)
            if (math.sqrt((x2-x1)**2 + (y2-y1)**2))<10: # This is to avoid that the robot spins when it achieves the goal point
                angle = 0.0
                LV = 0.0
            
            if case == 1:
                LV1 = LV
                AV1 = angle
                LV2 = 0.0
                AV2 = 0.0
                LV3 = 0.0
                AV3 = 0.0
            if case == 2:
                LV1 = 0.0
                AV1 = 0.0
                LV2 = LV
                AV2 = angle
                LV3 = 0.0
                AV3 = 0.0
            if case == 3:
                LV1 = 0.0
                AV1 = 0.0
                LV2 = 0.0
                AV2 = 0.0
                LV3 = LV
                AV3 = angle
            
            print(x2,y2)
            print("change",change)
#             if change == 1:
#                 LV1 = 0.0 
#                 AV1 = 0.0
#                 LV2 = 0.0
#                 AV2 = 0.0
#                 LV3 = 0.0
#                 AV3 = 0.0
#             LV1 = 0.0
#             angle = 0.0
#             clear_output()
            print(math.sqrt((x2-x1)**2 + (y2-y1)**2))
            print("Linear Velocity 1:", LV)
            print("Angular Velocity 1:", angle)
            print(arr[2],arr[3],arr[4])
            data = str(2)+ " " + str(LV1)+ " " + str(AV1) + "\n\0"
            #data = str(2)+ " " + str(0.0)+ " " + str(0.0) + "\n\0"
            data = str(data).encode()
            s.send(data)
            data = str(0)+ " " + str(LV3)+ " " + str(AV3) + "\n\0"
            #data = str(0)+ " " + str(0.0)+ " " + str(0.0) + "\n\0"
            data = str(data).encode()
            s.send(data)
            #data = str(1)+ " " + str(0.0)+ " " + str(0.0) + "\n\0"
            data = str(1)+ " " + str(LV2)+ " " + str(AV2) + "\n\0"
            data  = str(data).encode()
            s.send(data)
            print("FINISH")
    ########################################################################################################## ENDS REC DATA

Data from Python:
READY!
Key pressed
Reached the goal after 4.54 seconds:
Number of iterations passed: 165 / 5000
RRT length:  163
Constructing the path...
Reached RRT start node
Path smoothing...
[[1267.  757.]
 [1266.  754.]
 [1211.  667.]
 [1198.  647.]
 [1184.  629.]
 [1180.  624.]
 [1168.  614.]
 [1168.  614.]
 [1165.  614.]
 [1163.  613.]
 [1161.  613.]
 [1160.  613.]
 [1159.  613.]
 [1158.  613.]
 [1157.  612.]
 [1156.  612.]
 [1156.  612.]
 [1156.  612.]
 [1154.  612.]]
1267.0 757.0
change 1
1337.4369272771125
Linear Velocity 1: 93.62058490939789
Angular Velocity 1: -64.0410948056735
225.00 500.00 90.00
FINISH
1266.0 754.0
change 1
4.743416490252569
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1211.0 667.0
change 1
106.90094714267035
Linear Velocity 1: 7.483066299986925
Angular Velocity 1: 28.83957647110115
225.00 500.00 90.00
FINISH
1198.0 647.0
change 1
130.96278097230527
Linear Velocity 1: 9.16739466806137
Angular Velocity 1: 28.92705465993778
22

1154.0 612.0
change 1
56.212209527824115
Linear Velocity 1: 3.9348546669476883
Angular Velocity 1: 0.13155688408796776
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
55.45550017807071
Linear Velocity 1: 3.88188501246495
Angular Velocity 1: -0.3369760215433018
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
53.3391272894486
Linear Velocity 1: 3.733738910261402
Angular Velocity 1: 0.6785478590578634
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
50.51794235714673
Linear Velocity 1: 3.5362559650002714
Angular Velocity 1: -0.11149288635957645
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
49.391927478080866
Linear Velocity 1: 3.457434923465661
Angular Velocity 1: -0.7275267603453756
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
48.78332194510743
Linear Velocity 1: 3.4148325361575207
Angular Velocity 1: 1.1525303248991576
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
46.61343690396579
Linear Velocity 1: 3.2629405832776057
Angular Velocity 1: -0.16076384702844848
225.00 500.00 9

1154.0 612.0
change 1
10.550473923004596
Linear Velocity 1: 0.7385331746103218
Angular Velocity 1: 2.563904428727858
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0

1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.0

1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.0

...

1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50

1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50

1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50

1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50 499.50 90.00
FINISH
1154.0 612.0
change 1
9.017344398435716
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
225.00 500.00 90.00
FINISH
1154.0 612.0
change 1
9.38416218956173
Linear Velocity 1: 0.0
Angular Velocity 1: 0.0
224.50

ConnectionResetError: [WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto